In [ ]:
import logging
import pdb
logging.basicConfig(level=logging.INFO)
%matplotlib inline
%run LocalRepo.ipynb
%run repos.ipynb
%run parsing.ipynb
%run metrics.ipynb
%run analysis.ipynb
import pyfiglet
import numpy as np
# np.seterr(all='raise')
import os


repos = [
    "eclipse/eclipse.jdt.core",  # from duerschmidt
    "ErikBrendel/LudumDare", # 8 minutes BTM
    "neuland/jade4j",
    "jfree/jfreechart",
    "brettwooldridge/HikariCP",  # 1.2h BTM
    "adamfisk/LittleProxy",  # 1.5h BTM
    "dynjs/dynjs",  # 2.5h BTM
    # "SonarSource/sonarqube",  # quite big / prob. several hours BTM
    "square/okhttp",
    "jenkinsci/jenkins",
    "eclipse/che",
    "elastic/elasticsearch",
    "apache/camel"
    "jOOQ/jOOQ"
]
repos = ["eclipse/eclipse.jdt.core"]



for repo in repos:
    r = LocalRepo(repo)
    print(pyfiglet.figlet_format(r.name))
    r.update()
    
    # pdb.set_trace()
    print("Nodes in project structure:", r.get_tree().node_count())
    
    for view in ["structural"]: # ["structural", "evolutionary", "linguistic"]
        MetricManager.clear(r, view)

        coupling_graph = MetricManager.get(r, view)
        # coupling_graph.plaintext_save(repo)
        # coupling_graph.html_save(repo)
        

        print("\nResults from the " + view + " view:\n")

        coupling_graph.print_statistics()
        #coupling_graph.print_most_linked_nodes()
        #coupling_graph.show_weight_histogram()
        #coupling_graph.visualize(use_spring=False, with_labels=False)
        print("\n")
    
    # amalysis
    break
    repo_tree = r.get_tree()
    def node_filter(node_path):
        tree_node = repo_tree.find_node(node_path)
        return tree_node.get_type() == "method" and tree_node.get_line_span() >= 10
    def node_pair_filter(node_a_path, node_b_path):
        return path_module_distance(node_a_path, node_b_path) > 10
        
    #analyze_disagreements(r, ["structural", "evolutionary", "linguistic"], [
    #    [0, 1, None, "unrelated functionality that gets adjusted together"],
    #    [0, 0, 1, "duplicated concepts, developed separately"],
    #    [None, 0, 1, "duplicated concepts, adjusted separately"],
    #    [1, None, 0, "weakly modularized code"]
    #], node_filter, node_pair_filter)
    
    analyze_disagreements(r, ["structural", "evolutionary", "module_distance"], [
        [0, 1, None, "unrelated functionality that gets adjusted together"],
        [1, None, 0, "Strong structural coupling across a big distance"],
        [None, 1, 0, "Strong evolutionary coupling across a big distance"],
        [0, 1, 0, "Strong evolutionary and low structural coupling across a big distance"],
    ], node_filter, node_pair_filter)
    
    #analyze_disagreements(r, ["evolutionary", "module_distance"], [
    #    [1, 0, "Strong evolutionary coupling across a big distance"],
    #], node_filter, node_pair_filter)
    
    #analyze_disagreements(r, ["structural", "evolutionary"], [
    #    [0, 1, "adjusted together without structural relation"],
    #], node_filter, node_pair_filter)
    
print("\nProgram is over!")